# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

dt.timedelta(days = 1) -> 하루의 시간

dt.timedelta(days = 30) -> 30일의 시간

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# request가 정상적으로 이루어졌는지 확인 / 200은 정상
response.status_code

200

### Isolate the Data from the JSON Response
데이터의 구조를 살펴본다.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

여러 key값들 중 metadata는 우리가 찾는 정보가 아니다.

In [3]:
earthquake_json['metadata']

{'generated': 1690797788000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-06-30&endtime=2023-07-30',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.0',
 'count': 11367}

`features`가 어떤 유형의 데이터인지 확인.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [6]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.6,
  'place': '42 km ENE of Whittier, Alaska',
  'time': 1690674957393,
  'updated': 1690675053164,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak0239novrff',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak0239novrff&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 39,
  'net': 'ak',
  'code': '0239novrff',
  'ids': ',ak0239novrff,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.57,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.6 - 42 km ENE of Whittier, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-148.0183, 60.9817, 0]},
 'id': 'ak0239novrff'}

우리에게 필요한 데이터는 earthquake_json 딕셔너리 안의 **properties 딕셔너리 리스트이다.**

### Convert to DataFrame
 `properties` 부분만 빼내기 위해 리스트 컴프리헨션 사용.

In [7]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.60,"42 km ENE of Whittier, Alaska",1690674957393,1690675053164,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak0239novrff,",",ak,",",origin,phase-data,",NaN,NaN,0.57,NaN,ml,earthquake,"M 1.6 - 42 km ENE of Whittier, Alaska"
1,0.93,"12 km WSW of Alder, Washington",1690674100770,1690692777640,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",uw61936172,",",uw,",",origin,phase-data,",23.0,0.098400,0.20,73.0,ml,earthquake,"M 0.9 - 12 km WSW of Alder, Washington"
2,1.42,"16 km WSW of Lostine, Oregon",1690673912940,1690693415200,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",uw61936167,",",uw,",",origin,phase-data,",6.0,0.339600,0.27,79.0,ml,earthquake,"M 1.4 - 16 km WSW of Lostine, Oregon"
3,0.94,"11 km N of Borrego Springs, CA",1690672838740,1690749059211,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39625370,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",42.0,0.023900,0.16,67.0,ml,earthquake,"M 0.9 - 11 km N of Borrego Springs, CA"
4,0.81,"7 km WNW of Cobb, CA",1690672777060,1690674254140,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73918061,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",10.0,0.007352,0.01,93.0,md,earthquake,"M 0.8 - 7 km WNW of Cobb, CA"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>